In [1]:
import random
import math
import itertools
import logging
import numpy as np
import pandas as pd
from sklearn import datasets
import statsmodels.api as sm
from imageio import imread
import time
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim 
from geopy.distance import distance
from geopy import geocoders
logging.basicConfig(format = '%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

In [2]:
class SimulatedAnnealing(object):
    #applies simulated annealing to a TSP
    def _init_(self):
        self.temperature_initial = 1
        self.temperature_terminal = .0001
        self.cooling_rate = .99
        self.x = None
        self.y = None
        self.trip = None
        self.neighbor_trip = None
        self.state_evolution=[]
        self.distances = None
        self.latlong = None
        
    def define_problem(self, cities, round_trip=True, shuffle = True):
        """define the problem
        cities : list of name of cities as strings
        round_trip : true if start == end
        shuffle : choose random permutation"""
        if round_trip :
            self.trip = cities
            if shuffle :
                 random.shuffle(self.trip)
                    
    @staticmethod
    def retrieve_latlong_for_city(city) :
        """calculate distances for all the cities
        and give output as matrix"""
        geolocator = Nominatim(user_agent="TSP_Annealing")
        c = geolocator.geocode(city)
        return [c.latitude, c.longitude]
    
    def retrieve_distance_between_cities(self, city1, city2, unit = 'Km') :
        """to find distances betweeen cities"""
        geolocator = Nominatim(user_agent="TSP_Annealing")
        c1 = geolocator.geocode(city1)
        c2 = geolocator.geocode(city2)
        print("distance between "+city1 + "and " + city2, distance(c1.point,c2.point).km)
        if unit == 'km':
            return distance(c1.point,c2.point).km
        if unit == 'miles':
            return distance(c1.point,c2.point).miles
        
    def make_latlong(self, load_from_file = None):
        """generate a data frame using previously defined cities 
        with their respective longitude and latitude"""
        if load_from_file :
            self.latlong = pd.read_csv(load_from_file)
        else :
            assert self.trip is not None,'The problem has not been defined. define_problem()'
            latlong = pd.DataFrame(self.trip)
            latlong[1] = latlong[0].apply(t.retrieve_latlong_for_city)
            latlong[['latitude','longitude']] = pd.DataFrame([x for x in latlong[1]])
            latlong.index = latlong[0]
            del latlong[0]
            del latlong[1]
            self.latlong = latlong
            # print(latlong)
    def make_distances(self, load_from_file = None):
        if load_from_file:
            self.distances = pd.read_csv(load_from_file)
        else:
            assert self.trip is not None,'The problem has not been defined. define_problem()'
            distances = pd.DataFrame(np.zeros([len(self.trip),len(self.trip)]))
            # a matrix with zeros and order of length trip
            distances = distances.replace(0, -1)
            distances.columns = self.trip
            distances.index = self.trip
            self.distances = distances
            # column and row name alloted and zeros replaced with -1
            for city in self.trip:
                #print("Before")
                self.distances.loc[city].loc[city] = 0
            for city_tuple in itertools.combinations(self.trip, 2):
                logging.debug(city_tuple)
                if self.distances.loc[city_tuple[0]].loc[city_tuple[1]] < 0 \
                       or self.distances.loc[city_tuple[1]].loc[city_tuple[0]] < 0:
                    try:
                        # before calling retrieve_distance()
                        dist = self.retrieve_distance_between_cities(city_tuple[0], city_tuple[1])
                    except GeocoderUnavailable:
                        logging.warning('Geopy geocode was not available')
                        dist = np.nan
                    except HTTPError:
                        logging.warning('Too many equests for geopy, waitinf for a few sectonds')
                        time.sleep(30)
                        dist = np.nan
                    self.distances.loc[city_tuple[0]].loc[city_tuple[1]] = dist
                    self.distances.loc[city_tuple[1]].loc[city_tuple[0]] = dist
                    
    def energy(self, trip):
        """energy function for the annealing progress 
           calculate the total distance of the journey with the current order of the cities
           """
        return self.journey_distance(trip)
    def journey_distance(self, trip, round_trip = True):
        """Take a list of cities and calculate the distance of the round_trip"""
        assert self.distances is not None,'No distance matrix loaded'
        distances = []
        if round_trip:
            for i in range(len(trip)-1):
                distances.append(self.distances.loc[trip[i]].loc[trip[i+1]]) #------@1
        return np.array(distances).sum()
    @staticmethod
    def acceptance_probability(energy_old, energy_new, T):
        """compare energy states w.r.t. current temperaure"""
        return np.exp((energy_old - energy_new)/T)
    def generate_neighbor(self):
        """switch two random cities"""
        trip.neighbor = self.trip.copy()
        sw = random.sample(range(len(self.trip)), 2)
        trip_neighbor[sw[1]],trip_neighbor[sw[0]] = trip_neighbor[sw[0]], trip_neighbor[sw[1]]
        self.trip_neighbor = trip_neighbor
        
    def anneal(self):
        """simulated annealing"""
        assert self.distances is not None,'Distance file is not present'
        assert self.trip is not None,'problem is not defined, run self.define_problem()'
        self.state_evolution.append(self.trip) #state_evolution only serves to follow up on the states chosen
        T = self.temperature_initial
        while T > self.temperture_terminal:
            for n in range(len(self.trip)):
                self.generate_neighbor()
                energy_old = self.energy(self.trip)
                energy_new = self.energy(self.trip_neighbor)
                if self.acceptance_probability(energy_old, energy_new, T) > random.random():
                    self.trip = self.trip_neighbor #the alternative state was accepted
                    self.state_evolution.append(self.trip)
            T *= self.cooling_rate
        return self.trip
                           
    
    def energypath(self):
                           for n in range(len(self.state_evolution)):
                                print(self.state_evolution[n],"Distance:", self.journey_distance(self.state_evolution[n]))
    # Visualization
                           
    def visualize_one_state(self, state):
        """Draw one state as map"""
        if self.latlong is None:
             self.make_latlong()
        # city grid
        self.latlong.plot.scatter('longitude', 'latitude', zorder=1)
        for n in range(len(state)-1):
             self.draw_line_between(state[n], state[n+1])
        return plt
    
    def draw_line_between(self, city1, city2):
            """Draw a line between two cities"""
            plt.plot([self.latlong.loc[city1].loc['longitude'],self.latlong.loc[city2].loc['longitude']],
                     [self.latlong.loc[city1].loc['latitude'],self.latlong.loc[city2].loc['latitude']])
            return plt
    def visualize_all_states(self, folder=''):
            """save all different states as png"""
            i=0
            for state in self.state_evolution:
                    plt = self.visualize_one_state(state)
                    plt.savefig(folder+ 'tsp_'+ str(i))
                    i += 1
    def annotate_city(self, city):
            """Add city labels to the viualizations"""
            x_lab = self.latlong.loc[city].loc['longitude'] + 0.5
            y_lab = self.latlong.loc[city].loc['latitude'] + 0.5
            return plt.annotate(city, xy=(x_lab, y_lab), xytext=(x_lab, y_lab))
                           
                                                  
    
        

In [3]:
t = SimulatedAnnealing()

In [4]:
t.define_problem(['Chittorgarh', 'Udaipur', 'Ajmer', 'Pushkar', 'Ranthambore', 'Jaipur', 'Jodhpur', 'Bikaner', 'Bundi', 'Bharatpur', 'Kota', 'Mount Abu', 'Virat Nagar', 'Kumbhalgarh', 'Shekhawati', 'Sariska', 'Ranakpur', 'Pali', 'Karauli', 'Alwar'])

In [5]:
t.make_distances()

distance between Mount Abuand Shekhawati 454.4916113078932
distance between Mount Abuand Chittorgarh 179.10535955330576
distance between Mount Abuand Pali 132.85470073281996
distance between Mount Abuand Virat Nagar 1060.7401802963054
distance between Mount Abuand Ranthambore 409.12872567721007
distance between Mount Abuand Alwar 516.2426508663602


2021-02-26 14:03:38,824 : WARNING : Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=Mount+Abu&format=json&limit=1
2021-02-26 14:03:39,839 : WARNING : Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001F49D00DC40>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)')': /search?q=Mount+Abu&format=json&limit=1


distance between Mount Abuand Jaipur 404.4816721742949
distance between Mount Abuand Ajmer 284.3919589840875
distance between Mount Abuand Udaipur 99.07624422894371
distance between Mount Abuand Sariska 480.7071377982855
distance between Mount Abuand Pushkar 280.41609136707984
distance between Mount Abuand Karauli 486.25789810734256
distance between Mount Abuand Bundi 330.9906002199232


2021-02-26 14:03:52,030 : WARNING : Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=Mount+Abu&format=json&limit=1


distance between Mount Abuand Kota 339.3410586490227
distance between Mount Abuand Ranakpur 117.13015639202708


...

distance between Kotaand Bharatpur 266.8363732050709
distance between Kotaand Bikaner 411.07437559099003


2021-02-26 14:09:13,359 : WARNING : Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=Jodhpur&format=json&limit=1


distance between Ranakpurand Jodhpur 289.0100041673879
distance between Ranakpurand Kumbhalgarh 224.76934633732014
distance between Ranakpurand Bharatpur 669.680034627456
distance between Ranakpurand Bikaner 476.11525509951264
distance between Jodhpurand Kumbhalgarh 138.61686900425732


2021-02-26 14:09:21,459 : WARNING : Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=Bharatpur&format=json&limit=1
2021-02-26 14:09:22,479 : WARNING : Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001F4A3019FA0>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)')': /search?q=Bharatpur&format=json&limit=1


distance between Jodhpurand Bharatpur 444.12663492394915
distance between Jodhpurand Bikaner 192.53228035499112
distance between Kumbhalgarhand Bharatpur 445.175761364644
distance between Kumbhalgarhand Bikaner 318.8517675157842
distance between Bharatpurand Bikaner 408.4148185666955


In [6]:
t.distances

,Mount Abu,Shekhawati,Chittorgarh,Pali,Virat Nagar,Ranthambore,Alwar,Jaipur,Ajmer,Udaipur,Sariska,Pushkar,Karauli,Bundi,Kota,Ranakpur,Jodhpur,Kumbhalgarh,Bharatpur,Bikaner
Mount Abu,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Shekhawati,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chittorgarh,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pali,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Virat Nagar,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ranthambore,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alwar,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jaipur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ajmer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Udaipur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
t.make_latlong()

2021-02-26 14:10:38,246 : WARNING : Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=Shekhawati&format=json&limit=1
2021-02-26 14:10:42,780 : WARNING : Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001F4A30878E0>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)')': /search?q=Virat+Nagar&format=json&limit=1
2021-02-26 14:10:51,319 : WARNING : Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=Pushkar&format=json&limit=1
2021-02-26 14:10:56,370 : WARNING : Retry

In [8]:
t.latlong

,latitude,longitude
0,,
Mount Abu,24.592433,72.708188
Shekhawati,28.053891,75.151696
Chittorgarh,24.718026,74.472147
Pali,25.604091,73.415609
Virat Nagar,16.882033,78.760461
Ranthambore,26.018395,76.456306
Alwar,27.639077,76.614452
Jaipur,26.915458,75.818982
Ajmer,26.469100,74.639000


In [10]:
t.anneal()

AttributeError: 'SimulatedAnnealing' object has no attribute 'state_evolution'

In [11]:
t.energypath()

AttributeError: 'SimulatedAnnealing' object has no attribute 'state_evolution'

In [12]:
t.visualize_all_states()

AttributeError: 'SimulatedAnnealing' object has no attribute 'state_evolution'